# 🎓 Desafio 01: Web Scraping e Pipeline ETL

## 1\. Cenário e Fonte de Dados

Você recebeu uma string HTML bruta representando uma lista de produtos de um e-commerce. Seu objetivo é estruturar esses dados, aplicar regras de negócio (cálculo de preços) e padronizar formatos.

**Dados de Entrada (`HTML_DATA`):**

```html
<div class="product-list">
    <div class="product-card" id="P001" data-date="27/11/2025">
        <h2>Smartphone X10</h2>
        <img src="/img/x10.jpg" alt="X10">
        <p class="price">R$ 1.999,99</p>
        <span class="rating">4.5/5.0</span>
        <span class="discount-rate">10% OFF</span>
    </div>
    <div class="product-card" id="P002" data-date="28/11/2025">
        <h2>Notebook Ultraline</h2>
        <img src="/img/ultra.jpg" alt="Ultra">
        <p class="price">R$ 4.500,00</p>
        <span class="rating">4.8/5.0</span>
        </div>
    <div class="product-card" id="P003" data-date="28/11/2025">
        <h2>Fone Bluetooth Pro</h2>
        <img src="/img/fone.jpg" alt="Fone">
        <p class="price">R$ 549,50</p>
        <span class="rating">3.9/5.0</span>
        <span class="discount-rate">5% OFF</span>
    </div>
    <div class="product-card" id="P004" data-date="27/11/2025">
        <h2>Smartwatch Z</h2>
        <img src="/img/watch.jpg" alt="Watch">
        <p class="price">R$ 1.250,00</p>
        <span class="rating">5.0/5.0</span>
        <span class="discount-rate">20% OFF</span>
    </div>
</div>
```

## 2\. Especificações do Pipeline

### 2.1 Extração (Extract)

**Ferramenta recomendada:** `BeautifulSoup`

O objetivo desta fase é recuperar os dados brutos, aplicando um filtro de integridade rígido.

1.  Localize todos os cards de produtos.
2.  Extraia os campos `nome`, `preco`, `avaliacao`, `url_imagem`, `data` e `desconto`.
3.  Se um produto **não** possuir a tag de preço, ele deve ser **descartado imediatamente**. Não envie dados incompletos para a fase de Transformação.
4.  Imprima (log ou print) um alerta informando qual item foi descartado.

### 2.2 Transformação (Transform)

**Ferramenta recomendada:** `pandas`

Nesta fase, você deve limpar os tipos de dados e aplicar a lógica de negócio.

  * **Conversão Numérica:**
      * Transforme `preco` (Ex: "R$ 1.999,99") em `float` (`preco_numerico`).
      * Transforme `desconto` (Ex: "10% OFF") em `float` (`desconto_percentual`).
      * Normalize `avaliacao` para uma escala de 0 a 100 (`avaliacao_percentual`).
  * **Formatação de Data:**
      * Converta a data `dd/mm/yyyy` para `dd-mm-yyyy`.
  * **Cálculo do preço líquido:**
      * Calcule o preço final (preco_liquido) do produto, aplicando o desconto sobre o preço original.

### 2.3 Carga (Load)

**Saída:** Arquivo CSV

  * Gere um arquivo chamado **`ofertas_calculadas.csv`**.
  * O arquivo **não** deve conter o índice do DataFrame.
  * O separador deve ser vírgula (padrão).



## 3\. Resultado Esperado

Após a execução, seu DataFrame (e o CSV) deve apresentar os seguintes dados (note a ausência do *Notebook Ultraline* e o formato da data):

| nome | preco\_numerico | desconto\_percentual | **preco\_liquido** | avaliacao\_percentual | **data** |
| :--- | :--- | :--- | :--- | :--- | :--- |
| Smartphone X10 | 1999.99 | 10.0 | 1799.99 | 90.0 | 27-11-2025 |
| Fone Bluetooth Pro | 549.50 | 5.0 | 522.03 | 78.0 | 28-11-2025 |
| Smartwatch Z | 1250.00 | 20.0 | 1000.00 | 100.0 | 27-11-2025 |

# **Dados de Entrada**

In [6]:
HTML_DATA = """
<div class="product-list">
    <div class="product-card" id="P001" data-date="27/11/2025">
        <h2>Smartphone X10</h2>
        <img src="/img/x10.jpg" alt="X10">
        <p class="price">R$ 1.999,99</p>
        <span class="rating">4.5/5.0</span>
        <span class="discount-rate">10% OFF</span>
    </div>
    <div class="product-card" id="P002" data-date="28/11/2025">
        <h2>Notebook Ultraline</h2>
        <img src="/img/ultra.jpg" alt="Ultra">
        <p class="price">R$ 4.500,00</p>
        <span class="rating">4.8/5.0</span>
        </div>
    <div class="product-card" id="P003" data-date="28/11/2025">
        <h2>Fone Bluetooth Pro</h2>
        <img src="/img/fone.jpg" alt="Fone">
        <p class="price">R$ 549,50</p>
        <span class="rating">3.9/5.0</span>
        <span class="discount-rate">5% OFF</span>
    </div>
    <div class="product-card" id="P004" data-date="27/11/2025">
        <h2>Smartwatch Z</h2>
        <img src="/img/watch.jpg" alt="Watch">
        <p class="price">R$ 1.250,00</p>
        <span class="rating">5.0/5.0</span>
        <span class="discount-rate">20% OFF</span>
    </div>
</div>
"""

In [7]:
from bs4 import BeautifulSoup
import pandas as pd
import re
import os

# **Extração**

In [8]:
def extract_data(html_data):
    """
    Extrai dados dos cartões de produtos no HTML.
    Descarta produtos sem tag de preço, conforme a regra de integridade.
    """
    soup = BeautifulSoup(html_data, 'html.parser')
    product_cards = soup.find_all('div', class_='product-card')
    extracted_products = []

    for card in product_cards:
        # Recupera o ID do produto para alertas
        product_id = card.get('id', 'N/A')
        product_name = card.find('h2').text.strip() if card.find('h2') else 'N/A'

        # Regra de Integridade: Descartar se não houver preço
        price_tag = card.find('p', class_='price')
        if not price_tag:
            print(f"!!! Alerta: Item '{product_name}' (ID: {product_id}) descartado por falta da tag de preço.")
            continue  # Pula este item e vai para o próximo

        # Extração dos demais campos
        price_raw = price_tag.text.strip()

        rating_tag = card.find('span', class_='rating')
        rating_raw = rating_tag.text.strip() if rating_tag else 'N/A'

        img_tag = card.find('img')
        img_url = img_tag.get('src') if img_tag else 'N/A'

        date_raw = card.get('data-date', 'N/A')

        discount_tag = card.find('span', class_='discount-rate')
        # Se não houver desconto, assume-se 0% para facilitar a transformação
        discount_raw = discount_tag.text.strip() if discount_tag else '0% OFF'

        # Armazena os dados brutos (strings)
        extracted_products.append({
            'nome': product_name,
            'preco': price_raw,
            'avaliacao': rating_raw,
            'url_imagem': img_url,
            'data': date_raw,
            'desconto': discount_raw
        })

    return pd.DataFrame(extracted_products)

# Execução da Extração
df_extracted = extract_data(HTML_DATA)
print("DataFrame após Extração (Dados Brutos):")
display(df_extracted)


DataFrame após Extração (Dados Brutos):


,nome,preco,avaliacao,url_imagem,data,desconto
0,Smartphone X10,"R$ 1.999,99",4.5/5.0,/img/x10.jpg,27/11/2025,10% OFF
1,Notebook Ultraline,"R$ 4.500,00",4.8/5.0,/img/ultra.jpg,28/11/2025,0% OFF
2,Fone Bluetooth Pro,"R$ 549,50",3.9/5.0,/img/fone.jpg,28/11/2025,5% OFF
3,Smartwatch Z,"R$ 1.250,00",5.0/5.0,/img/watch.jpg,27/11/2025,20% OFF


# **Transformação**

In [9]:
def transform_data(df):
    """
    Limpa, converte e calcula os campos de negócio.
    """

    # Conversão Numérica

    # Função para limpar e converter preço
    def clean_price(price_str):
        # Remove 'R$' e substitui ponto por nada e vírgula por ponto para float
        cleaned = price_str.replace('R$', '').replace('.', '').replace(',', '.').strip()
        return float(cleaned)

    # Aplica a limpeza do preço
    df['preco_numerico'] = df['preco'].apply(clean_price)

    # Função para limpar e converter desconto para o formato: 00.0)
    def clean_discount(discount_str):
        # Extrai o número no início da string
        match = re.search(r'(\d+(\.\d+)?)', discount_str)
        return float(match.group(0)) if match else 0.0 # Retorna 0.0 se não encontrar

    # Aplica a limpeza do desconto
    df['desconto_percentual'] = df['desconto'].apply(clean_discount)

    # Função para normalizar avaliação
    def normalize_rating(rating_str):
        if '/' in rating_str:
            # Assume sempre x/5.0, mas extrai os números de forma mais robusta
            parts = re.findall(r'(\d+\.?\d*)', rating_str)
            if len(parts) >= 2 and float(parts[1]) > 0:
                score = float(parts[0])
                max_score = float(parts[1])
                return (score / max_score) * 100
        return 0.0

    # Aplica a normalização da avaliação
    df['avaliacao_percentual'] = df['avaliacao'].apply(normalize_rating)

    #  Formatação de Data

    # Converte data de 'dd/mm/yyyy' para 'dd-mm-yyyy'
    df['data'] = df['data'].str.replace('/', '-', regex=False)

    #  Cálculo do Preço Líquido

    # Fórmula: preco_liquido = preco_numerico * (1 - desconto_percentual / 100)
    df['preco_liquido'] = df.apply(
        lambda row: round(row['preco_numerico'] * (1 - row['desconto_percentual'] / 100), 2),
        axis=1
    )

    # Seleciona e reordena as colunas finais
    df_transformed = df[[
        'nome',
        'preco_numerico',
        'desconto_percentual',
        'preco_liquido',
        'avaliacao_percentual',
        'data'
    ]]

    return df_transformed

# Execução da Transformação
df_transformed = transform_data(df_extracted.copy())
print("DataFrame após Transformação (Dados Finais):")
#print(df_transformed)
display(df_transformed)

DataFrame após Transformação (Dados Finais):


,nome,preco_numerico,desconto_percentual,preco_liquido,avaliacao_percentual,data
0,Smartphone X10,1999.99,10.0,1799.99,90.0,27-11-2025
1,Notebook Ultraline,4500.00,0.0,4500.00,96.0,28-11-2025
2,Fone Bluetooth Pro,549.50,5.0,522.02,78.0,28-11-2025
3,Smartwatch Z,1250.00,20.0,1000.00,100.0,27-11-2025


# **Carga (Load)**

In [10]:
def load_data(df, filename="ofertas_calculadas.csv"):
    """
    Carrega o DataFrame em um arquivo CSV.
    """
    # Cria o arquivo CSV, sem o índice do DataFrame
    df.to_csv(filename, index=False, sep=',')

    # Confirmação
    absolute_path = os.path.abspath(filename)
    print(f"Dados carregados com sucesso no arquivo: **{filename}**")
    print(f"Localização absoluta: {absolute_path}")

# Execução da Carga
load_data(df_transformed)

Dados carregados com sucesso no arquivo: **ofertas_calculadas.csv**
Localização absoluta: /content/ofertas_calculadas.csv
